Myopia Clusters!

In [ ]:
#Imports
import pandas as pd
from sklearn.cluster import KMeans
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
file_path = "Resources/myopia.csv"
my_df = pd.read_csv(file_path)

In [ ]:
my_df

In [ ]:
myopia_df = my_df.drop(["MYOPIC"], axis=1)
myopia_df

In [ ]:
#scale the columns in order for them to be closer in range with eachother
scaler = StandardScaler()
scaled_data = scaler.fit_transform(myopia_df[["AGE", "SPHEQ", "AL", "ACD", "LT", "VCD", "SPORTHR", 
                                              "READHR", "COMPHR", "STUDYHR", "TVHR", "DIOPTERHR", "MOMMY", "DADMY"]])

In [ ]:
scaled_df = pd.DataFrame(scaled_data, columns=myopia_df.columns)
scaled_df

In [ ]:
#Add PCA elements
pca = PCA(n_components=.9)
myopia_PCA = pca.fit_transform(scaled_df)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
df_myopia_pca = pd.DataFrame(
    data=myopia_PCA, columns=["PC_1", "PC_2", "PC_3", "PC_4", "PC_5", "PC_6", "PC_7", "PC_8", "PC_9", "PC_10"])
df_myopia_pca.head()

In [ ]:
#add a t-sne element
from sklearn.manifold import TSNE
tsne = TSNE(learning_rate=50)
tsne_features = tsne.fit_transform(df_myopia_pca)

In [ ]:
tsne_features.shape

In [ ]:
df_myopia_pca['PC_x'] = tsne_features[:,0]
df_myopia_pca["PC_y"] = tsne_features[:,1]


In [ ]:
#Plot the results from the TSNE
plt.scatter(df_myopia_pca['PC_x'], df_myopia_pca["PC_y"])
plt.show()

Based on the above scatter plot of the TSNE results, it appears there are roughly 4 different clusters. 

In [ ]:
#add labels
labels=my_df["MYOPIC"]

In [ ]:
plt.scatter(df_myopia_pca['PC_x'], df_myopia_pca["PC_y"], c=labels)
plt.show()

In [ ]:
#Create and runt the K-means model
model = KMeans(n_clusters=10, random_state=0)

In [ ]:
#fit the model
model.fit(df_myopia_pca)

In [ ]:
#predictions
predictions = model.predict(df_myopia_pca)

In [ ]:
#add a new class column to the dataset
df_myopia_pca["class"] = model.labels_
df_myopia_pca.head()

In [ ]:
#Find the best value for K
inertia = []
k = list(range(1, 11))

#look for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_myopia_pca)
    inertia.append(km.inertia_)
    
#define DF to plot Elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

plt.plot(df_elbow['k'], df_elbow['inertia'])
plt.xticks(list(range(11)))
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
df_elbow

In [ ]:
model = KMeans(n_clusters=3, random_state=4)
model.fit(df_myopia_pca)

In [ ]:
predicition = model.predict(df_myopia_pca)

In [ ]:
def get_clusters(k, data):
    # Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)

    # Train the model
    model.fit(df_myopia_pca)

    # Predict clusters
    predictions = model.predict(df_myopia_pca)

    # Create return DataFrame with predicted clusters
    df_myopia_pca["class"] = model.labels_

    return df_myopia_pca

In [ ]:
df_clusters = get_clusters(3, df_myopia_pca)
df_clusters

In [ ]:
def show_clusters(df):
    plt.scatter(df_myopia_pca['PC_x'], df_myopia_pca["PC_y"], c=df_myopia_pca["class"])
    plt.show
    
show_clusters(df_clusters)

After creating an unsupervised model for the myopia data, it appears the most accurate number of clusters for patients is 3. 